In [1]:
# 데이터 시각화
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

# 앙상블, Ensemble
- 여러개의 분류 모델을 조합해 더 나은 성능을 내는 방법
- Decision Tree 모델을 향상시킨 RandomForest가 대표적 사례이다.

---
### 랜덤포레스트(Random Forest)
- 부트스트랩 샘플을 사용합니다. 부트스트랩 샘플링은 중복을 허용하는 샘플링 방법입니다.
- 과대적합을 방지하기 위해, 샘플링이 반복될 때 제외하고 뽑는 것이 아닌 전체에서 다시 뽑는 방법을 사용

In [2]:
import pandas as pd

In [3]:
wine = pd.read_csv('../data/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
# Feature, Target
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
# 전체 셋 중에 훈련과 테스트 셋을 8:2의 기준으로 분리한다.
from sklearn.model_selection import train_test_split

In [6]:
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [7]:
# 랜덤포레스트 모델 구현
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [8]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42) # n_jobs=-1, 하드웨어의 자원을 모두 사용하겠다
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.32144165, 0.32442117, 0.40455818, 0.5253849 , 0.32933807]),
 'score_time': array([0.03219604, 0.03256893, 0.03258801, 0.04127097, 0.02111483]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

In [9]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [10]:
# 주요 Feature
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


> sugar가 주긴 하지만 의사결정나무보단 값이 적어졌다.

In [11]:
# 부트스트랩 결정시 남은 샘플(oob : out of back)로도 특성을 구분할 수 있다.
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


> oob로도 train test의 역할이 가능하다.

---
# Extra Tree
- 기본적으로 100개의 트리를 사용한다.
- 노드 분할시 특성의 제곱근의 개수를 사용한다.
- 특성의 선택을 랜덤하게 한다.
- 특성의 선택을 랜덤하게 하므로 속도는 랜덤포레스트보다 빠르다.

In [12]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [13]:
# 주요 Feature
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


> 역시 sugar가 주요하다.

---
### Gradient Boosting
- 가장 유명한 알고리즘 중 하나이다.
- 경사 하강법처럼 손실함수를 사용한다.
- 손실함수를 보고 트리를 추가하여 최적의 값을 도출하는 방법이다.
- Decision Tree Regressor를 사용하여 손실함수를 계산하고 이를 계속 낮추기 위해 트리를 추가하는 구조(max_depth 추가)
- 경사를 이동하면서 경사의 이동거리를 제어하는 learning-rate(기본 0.1)를 사용한다.
- max_depth를 3으로 제어하여 과대적합을 방지한다.(깊이가 낮으므로)
- 단점 : 손실함수를 보고 트리를 추가하며 진행하는 모델이므로 병렬처리(n_jobs)를 할 수 없다.

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

In [15]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [16]:
# 주요 Feature
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.11949946 0.74871836 0.13178218]


In [17]:
# 기본값을 변경 : tree 500, learning_rate 0.2
gb = GradientBoostingClassifier(random_state=42, n_estimators=500, learning_rate=0.2)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


> 5배 이상 올린 것 치고 잘 규제하고 있음

In [18]:
# 주요 Feature
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


---
### 히스토그램 기반 그래디언트 부스팅(Histogram Gradient Boosting)
- 훈련데이터를 256개의 구간으로 나누어서 훈련시키는 방법
- 특성의 범위가 제한되어 빠른 속도를 제공한다.
- 제한된 구간이므로 과대적합을 방지한다.
- 아직은 실험단계인 모델

In [19]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


# *번외*
### Permutation Importance, 치환 중요도
- 각 특성(Feature) 별 샘플을 섞어서 계산을 한 후 원래 sample과의 차이를 계산해서 차이가 많이날수록 중요하다고 판단

In [20]:
# Train 확인
from sklearn.inspection import permutation_importance
# 모델은 아무거나
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


> sugar인 경우 sample을 무작위로 섞으면 23%만큼 정확도가 떨어진다.   
> 다른 컬럼은 8% 정도 떨어지므로 중요하지 않다.

In [21]:
# Test 확인
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


> sugar만 20% 정도로 유의미하게 정확도가 떨어진다.

---
### XGBoost
- kaggle에서 많이 사용

In [22]:
# !pip install xgboost

In [23]:
from xgboost import XGBClassifier

In [26]:
xgb = XGBClassifier(
    tree_method='hist', # 히스토그램 그래디언트 부스팅
    random_state=42,
    use_label_encoder=False, 
    eval_metric='logloss' # 로지스틱 손실함수
    )
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


---
#### LightGBM

In [27]:
!pip install lightgbm

     |████████████████████████████████| 1.2 MB 11.0 MB/s eta 0:00:01


In [29]:
from lightgbm import LGBMClassifier

In [30]:
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


---
# 앙상블 정리
- 앙상블 학습은 정형데이터에서 가장 뛰어난 성능을 내는 머신러닝 알고리즘입니다.

### 랜덤 포레스트
: 부트스트랩 사용, 대표 앙상블 학습 알고리즘

### 엑스트라 트리
: 결정트리 노드를 랜덤하게 분할함

### 그래디언트 부스팅
: 결정트리의 손실을 보완하는 식으로 '얕은' 결정트리를 연속하여 추가함

### 히스토그램 기반 그래디언트 부스팅
: 훈련데이터를 256개의 정수 구간으로 나누어 빠르고 높은 성능을 냄

### 기타
- XGBoost
- LightGBM